In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
from rubric_assessment_app import app
app()


Token Valid: Alison Myers (185698)

